In [0]:
import pandas as pd
import os
import sys
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
num_cores = multiprocessing.cpu_count()
print(num_cores)
%matplotlib inline

2


In [0]:
df = pd.read_excel('w.xlsx', sep = '|', encode = 'utf-8')

In [0]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [0]:
class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[2]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[1]))
        print(prestr(val[2]))

наука
наука-система знаний, полученных в результате этой деятельности и составляющих научную картину мира 
учение
Совокупность теоретических положений о какой-н. области явлений действительности.
представление о конце чего-либо
Религиозное учение о загробной жизни человека, о конце света, входящее составной частью во многие религии.
полка
напольное или навесное изделие мебели.
разновидность стеллажа
полки которого открыты со всех сторон и опираются на вертикальные стойки
биплан
 самолёт, имеющий две несущих плоскости, расположенные одна над другой
место
часть пространства
напольное или навесное изделие мебели
напольное или навесное изделие мебели
органическое химическое соединение
органическое соединение 
бесцветный газ
бесцветный газ
продукт нефтехимической промышленности
продукт нефтехимической промышленности
мономер
 химическое соединение
газообразный растительный гормон
является одним из базовых продуктов промышленной химии и стоит в основании ряда цепочек синтеза
раздел медицины
и

In [0]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
p = morph.parse('акт')[0]
p.tag.POS

'NOUN'

In [0]:
!pip install yargy
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [0]:
from yargy import rule, or_

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN'),
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

START_N = rule(
    gram('NOUN'),
)

START_T = rule(
    or_(
        rule(gram('PREP')),
        rule(gram('NOUN'))
    ).optional()
)

KAK = eq('как')

INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),
)

INCLUDING_S = or_(
    eq('называют'),
    eq('называется'),
    eq('называть')
)


MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
    )

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или')
        ),
        eq('другие')
    ),
    rule(
        eq('—'),
        eq('то'),
        eq('есть'),
        eq(',')
    )
)

MID_N = or_(
    rule(
        eq('есть')
    ),
    rule(
        eq('есть'),
        eq('и'),
        eq('другое'),
        eq('значение'),
        eq('-'),
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('PREP'),
        gram('ADJF'),
        gram('NPRO')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('NPRO'),
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('CONJ').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('VERB'),
        gram('ADVB'),
        gram('ADJF')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')        
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()        
    ),
    rule(
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('VERB'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        eq('-'),
        gram('CONJ'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('PREP'),
        gram('NPRO'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF')
    ),
    rule(
        gram('PRCL'),
        gram('PRTF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADVB'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF')
    ),
    rule(
        gram('CONJ'),
        gram('ADVB'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('NPRO'),
        gram('NOUN').repeatable(),
    ),
    rule(
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF').repeatable()
    ),
    rule(
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN'),
        gram('VERB'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('ADJF'),
        gram('CONJ'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF'),
        gram('ADJF').repeatable(),
    ),
    rule(
        gram('ADJF'),
        gram('INFN'),
        gram('CONJ'),
        gram('INFN'),
    ),
    rule(
        gram('CONJ'),
        gram('NOUN'),
        gram('VERB'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('PREP'),
        gram('ADJF'),
        gram('VERB'),
        eq('16'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        eq('-'),
        eq('л'),
        eq('.'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        eq('-'),
        gram('CONJ'),
        gram('NOUN'),
        gram('CONJ'),
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('CONJ'),
        gram('ADVB'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN')
    ),
    rule(
        gram('PRTS')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('NPRO'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable(),
        gram('ADVB'),
        gram('VERB'),
        gram('PREP'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        gram('ADVB'),
        gram('VERB')
    ),
    rule(
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN')
    ),
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonym(i):
    main_word = (df['words'][i]).lower()
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(HYPONYM, MID, END, eq(','), END, eq(','), END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(START, MID, END),
        rule(START_N, INCLUDING_S, END),
        rule(HYPONYM, MID, eq('.'), END),
        rule(START_N, MID, END),
        rule(HYPONYM, MID, END, eq(','), END),
        rule(START_T, HYPONYM, MID_N, END),
        rule(HYPONYM, START, MID, END),
        rule(HYPONYM, MID, END, eq(','), END, eq(','), END),
        rule(END, eq(','), END, eq(','), INCLUDING_S, START_N),
        rule(END, END, START_N, KAK, END, eq(','), END),
        rule(END, START_N, KAK, END),
        rule(END, START_N, INCLUDING_S, END),
        rule(HYPONYM, eq(','), END, eq(','), END, eq(';'), END, eq(','), END)
)
    parser = Parser(RULE) 
    text = df['text'][i]
    text = re.sub(r'\(.+?\)', '', text)
    #for deleting "
    text = re.sub(r'"', '', text)
    print(text)
    hyperonyms = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyperonyms.append(k)
        print(k)
    print('-----------------------------------------')
    print(' ')

In [0]:

for i in range(len(df)):
    get_hyperonym(i)

наука-система знаний, полученных в результате этой деятельности и составляющих научную картину мира 
-----------------------------------------
 
учение-совокупность теоретических положений о какой-н. области явлений действительности.
-----------------------------------------
 
эсхатология-религиозное учение о загробной жизни человека, о конце света, входящее составной частью во многие религии.
-----------------------------------------
 
напольное или навесное изделие мебели.
-----------------------------------------
 
полки которого открыты со всех сторон и опираются на вертикальные стойки
-----------------------------------------
 
 самолёт, имеющий две несущих плоскости, расположенные одна над другой
-----------------------------------------
 
часть пространства
-----------------------------------------
 
напольное или навесное изделие мебели
-----------------------------------------
 
органическое соединение 
-----------------------------------------
 
бесцветный газ
---------------

TypeError: ignored